In [1]:
import Pkg 
Pkg.activate("..")

using Enzyme, Test, KernelAbstractions, CUDAKernels, CUDA, KernelGradients, BenchmarkTools, LinearAlgebra, Adapt, Parameters, SpeedyWeather

  Activating project at `~/Nextcloud/SpeedyExperiments/scripts`


# KernelAbstractions and Enzyme with SpeedyWeather

Now, we are going to start to work on SpeedyWeather.jl

For that, we also already defined some basic structure, you can find it in the `gpu.jl` file of SpeedyWeather.jl, but we'll introduce the most important bits here as well. The version we are working on here is the `kernelabstractions_adtests` branch of SpeedyWeather.jl.

First of all, we save information about the device we are using (so either a CPU or GPU) in the `DeviceSetup` struct.

In [2]:
?SpeedyWeather.DeviceSetup

```
DeviceSetup{S<:AbstractDevice}
```

Holds information about the device the model is running on and workgroup size. 

  * `device::AbstractDevice`: Device the model is running on
  * `device_KA::KernelAbstractions.Device`: Device for use with KernelAbstractions
  * `n`: workgroup size


For the local tests, we set it to CPU:

In [3]:
const device = SpeedyWeather.DeviceSetup(SpeedyWeather.CPUDevice())

SpeedyWeather.DeviceSetup{SpeedyWeather.CPUDevice, DataType}(SpeedyWeather.CPUDevice(), CPU, 4)

Additionally, if needed, we have some wrapper functions that, depending on the device choose the correct array format 

In [4]:
?SpeedyWeather.DeviceArray

```
DeviceArray(device::AbstractDevice, x)
```

Adapts `x` to a `CuArray` when `device<:GPUDevice` is used, otherwise a regular `Array`. Uses `adapt`, thus also can return SubArrays etc.


Given, a `DeviceSetup` we also have a handy way of launching KernelAbstractions kernels  

In [5]:
?SpeedyWeather.launch_kernel!

```
launch_kernel!(device_setup::DeviceSetup, kernel!, ndrange, kernel_args...)
```

Launches the `kernel!` on the `device_setup` with `ndrange` computations over the kernel and arguments `kernel_args`. Returns an event.


## A few comments to Speedy's internals 

Speedy is a spectral model. This means most of the computations are performed in the spherical harmonics domain which the variables being expansion coefficient of the spherical harmonics series expansion. For the  coefficients $(l,m)$, $m<=l$ is always valid. This is why one way of representing them would be the lower triangle of a matrix. In SpeedyWeather, we have a specialized format for this `LowerTriangularMatrix`

In [6]:
?SpeedyWeather.LowerTriangularMatrix

```
L = LowerTriangularMatrix{T}(v::Vector{T},m::Int,n::Int)
```

A lower triangular matrix implementation that only stores the non-zero entries explicitly. `L<:AbstractMatrix` although in general we have `L[i] != Matrix(L)[i]`, the former skips zero entries, tha latter includes them.

---

```
L = LowerTriangularMatrix(M)
```

Create a LowerTriangularMatrix `L` from Matrix `M` by copying over the non-zero elements in `M`.


which we can also use to iterate over directly with just one index in a for loop, as you will see in just a bit. Aside from that some variables are also applied in grid space where are typically doing a double for loop over the latitudes and longitudes. 

Speedy computes the atmosphere for several layers at different altitudes. Most computations within Speedy are done on each of these layers seperately. Most variables are therefore internally saved as `Vector{SomethingPerLayer}`


But, now let's look at a first example

# 1 - Horizontal Diffusion 

As a fist example, I kind of randomly picked the horizontal diffusion struct / function. 

But first, we have to load the enviroment, also load a state of SpeedyWeather that we can use 

In [7]:
progn_vars, diagn_vars, model_setup = initialize_speedy();

Per default, we get a barotropic model: 

In [8]:
model_setup

SpeedyWeather.BarotropicModel{Float32, SpeedyWeather.CPUDevice}(SpeedyWeather.Parameters
  NF: Float32 <: AbstractFloat
  model: Symbol barotropic
  grid: Symbol regular_Gaussian
  trunc: Int64 31
  nlev: Int64 1
  radius_earth: Float64 6.371e6
  rotation_earth: Float64 7.29e-5
  gravity: Float64 9.81
  akap: Float64 0.2857142857142857
  cp: Int64 1004
  R_gas: Float64 286.85714285714283
  alhc: Int64 2501
  alhs: Int64 2801
  sbc: Float64 5.67e-8
  lapse_rate: Int64 6
  temp_ref: Int64 288
  temp_top: Int64 216
  scale_height: Float64 7.5
  pres_ref: Int64 1013
  scale_height_humid: Float64 2.5
  relhumid_ref: Float64 0.7
  water_pres_ref: Int64 17
  layer_thickness: Int64 10
  GLcoefs: SpeedyWeather.GenLogisticCoefs{Float64}
  n_stratosphere_levels: Int64 2
  diffusion_power: Int64 4
  diffusion_time: Float64 2.4
  diffusion_time_div: Float64 2.4
  diffusion_time_strat: Int64 12
  damping_time_strat: Int64 720
  seasonal_cycle: Bool true
  n_shortwave: Int64 3
  sppt_on: Bool false
 

So, we want to look into rewriting/adjusting `horizontal_diffusion!` here as an example. First, we will look up how this function is called given an initialized model. Looking up the source code we can see that the function signature is 

```julia
function horizontal_diffusion!( tendency::LowerTriangularMatrix{Complex{NF}},   # tendency of a 
                                A::LowerTriangularMatrix{Complex{NF}},          # spectral horizontal field
                                damp_expl::LowerTriangularMatrix{NF},           # explicit spectral damping
                                damp_impl::LowerTriangularMatrix{NF}            # implicit spectral damping
                                ) where {NF<:AbstractFloat}
```

and it is called in the timestepping routine 

```julia

    # LOOP OVER LAYERS FOR DIFFUSION, LEAPFROGGING AND PROPAGATE STATE TO GRID
    for (progn_layer,diagn_layer) in zip(progn.layers,diagn.layers)
        get_tendencies!(diagn_layer,M)                      # tendency of vorticity
        horizontal_diffusion!(progn_layer,diagn_layer,M)    # diffusion for vorticity
        leapfrog!(progn_layer,diagn_layer,dt,lf1,M)         # leapfrog vorticity forward
        gridded!(diagn_layer,progn_layer,lf2,M)             # propagate spectral state to grid
    end
```


via this intermediate function: 

```julia

function horizontal_diffusion!( progn::PrognosticVariablesLeapfrog,
                                diagn::DiagnosticVariablesLayer,
                                M::BarotropicModel,
                                lf::Int=1,                          # leapfrog index used (2 is unstable)
                                )

    @unpack damping, damping_impl = M.horizontal_diffusion
    @unpack vor = progn.leapfrog[lf]
    @unpack vor_tend = diagn.tendencies

    horizontal_diffusion!(vor_tend,vor,damping,damping_impl)    # diffusion of vorticity
end

```

in which `progn` is an instance of the `PrognosticVariables` like the `progn_vars` we intialized, `diagn` is an instance of `DiagnosticVariables` like the `diagn_vars` we initialized and `M` is the `model_setup`  


Let's call it once, so we can later also cross-check our new version later. We have to explicitly add `SpeedyWeather` a few times as those functions are not exported (but only in this notebook, not later in the actual package).

In [9]:
M = model_setup 
diagn = diagn_vars.layers[1]
progn = progn_vars.layers[1] 
lf = 1
NF = Float32

SpeedyWeather.get_tendencies!(diagn, M)   

@unpack vor = progn.leapfrog[lf]
@unpack vor_tend = diagn.tendencies
@unpack damping, damping_impl = M.horizontal_diffusion
                                      
# we have to convert everything to CuArrays, incase we are on GPU 

vor_tend = SpeedyWeather.DeviceArray(device, vor_tend)
vor = SpeedyWeather.DeviceArray(device, vor)
damping = SpeedyWeather.DeviceArray(device, damping)
damping_impl = SpeedyWeather.DeviceArray(device, damping_impl)

SpeedyWeather.horizontal_diffusion!(vor_tend,vor,damping,damping_impl)

Now, that we know how to call the function, let's rewrite it to work on GPU and with Enzyme! 

First, we inspect the old version: 


```julia 
function horizontal_diffusion!( tendency::LowerTriangularMatrix{Complex{NF}},   # tendency of a 
                                A::LowerTriangularMatrix{Complex{NF}},          # spectral horizontal field
                                damp_expl::LowerTriangularMatrix{NF},           # explicit spectral damping
                                damp_impl::LowerTriangularMatrix{NF}            # implicit spectral damping
                                ) where {NF<:AbstractFloat}
    
    for lm in eachharmonic(tendency,A,damp_expl,damp_impl)
        @inbounds tendency[lm] = (tendency[lm] - damp_expl[lm]*A[lm])*damp_impl[lm]
    end
end
```

`eachharmonic` just returns a unit range over all elements of the `LowerTriangularMatrix`, but also performs some bounds checks. We can't use `eachharmonic` within the kernels, so we have to do the bounds checks again in the function as well. 

In [10]:
?SpeedyWeather.eachharmonic

```
unit_range = eachharmonic(L::LowerTriangular)
```

creates `unit_range::UnitRange` to loop over all non-zeros in a LowerTriangularMatrix `L`. Like `eachindex` but skips the upper triangle with zeros in `L`.

---

```
unit_range = eachharmonic(Ls::LowerTriangularMatrix...)
```

creates `unit_range::UnitRange` to loop over all non-zeros in the LowerTriangularMatrices provided as arguments. Checks bounds first. All LowerTriangularMatrixs need to be of the same size. Like `eachindex` but skips the upper triangle with zeros in `L`.


So, `horizontal_diffusion!` just consists of bounds checks and a for loop. We'll have to write a kernel for the loop and then call this kernel from a wrapper function that also includes the bounds checks 

Great! So, now let's write the horizontal diffusion kernel. In this case we are working with a `LowerTriangularMatrix` and loop only over one index, that's why we use `lm = @index(Global, Linear)`, in other cases we would need `i, j = @index(Global, NTuple)`. In this case the only array that we actually assign new values is `tendency`, all other are therefore `@Const`. 



In [11]:
@kernel function horizontal_diffusion_kernel!(tendency, @Const(A), @Const(damp_expl), @Const(damp_impl))
    lm = @index(Global, Linear)
   
    tendency[lm] = (tendency[lm] - damp_expl[lm]*A[lm])*damp_impl[lm]
end

horizontal_diffusion_kernel! (generic function with 5 methods)

In [12]:
length(vor)

560

We take the bounds checks from the old version an integrate now the kernel and launch it with `launch_kernel!` after adding `device_setup` to the input argument. `launch_kernel!` also needs an `ndrange` which tells the full range the kernel should loop over. In this case we have `length(tendency)` elements to loop over. In the case of a two-dimensional / double loop, we would have to use the appropiate `size(....)` that the loop is looping over. 

In [27]:
function horizontal_diffusion!( tendency::LowerTriangularMatrix{Complex{NF}},   # tendency of a 
                                A::LowerTriangularMatrix{Complex{NF}},          # spectral horizontal field
                                damp_expl::LowerTriangularMatrix{NF},           # explicit spectral damping
                                damp_impl::LowerTriangularMatrix{NF},            # implicit spectral damping
                                device_setup::SpeedyWeather.DeviceSetup,                      # device the function is executed on, we have to add that
                                ) where {NF<:AbstractFloat}

    @boundscheck size(A) == size(tendency) || throw(BoundsError())
    @boundscheck size(A) == size(damp_expl) || throw(BoundsError())
    @boundscheck size(A) == size(damp_impl) || throw(BoundsError())

    ev = SpeedyWeather.launch_kernel!(device_setup, horizontal_diffusion_kernel!, length(tendency), tendency, A, damp_expl, damp_impl)
    wait(ev)

end 


horizontal_diffusion! (generic function with 1 method)

Now we have to test if it works! We'll compare the old version to the KernelAbstractions version. 

In [28]:
vor_tend_old = deepcopy(vor_tend)
vor_tend_new = deepcopy(vor_tend)

SpeedyWeather.horizontal_diffusion!(vor_tend_old, vor, damping, damping_impl)
horizontal_diffusion!(vor_tend_new, vor, damping, damping_impl, device)

In [29]:
@test vor_tend_old ≈ vor_tend_new

Test Passed
  Expression: vor_tend_old ≈ vor_tend_new
   Evaluated: ComplexF32[-0.0f0 - 0.0f0im 0.0f0 + 0.0f0im … 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.0f0 - 0.0f0im -0.0f0 - 0.0f0im … 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im; … ; -0.0f0 - 0.0f0im -0.0f0 - 0.0f0im … -0.0f0 - 0.0f0im -0.0f0 - 0.0f0im; 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im … 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im] ≈ ComplexF32[-0.0f0 - 0.0f0im 0.0f0 + 0.0f0im … 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.0f0 - 0.0f0im -0.0f0 - 0.0f0im … 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im; … ; -0.0f0 - 0.0f0im -0.0f0 - 0.0f0im … -0.0f0 - 0.0f0im -0.0f0 - 0.0f0im; 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im … 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im]

Great! 

Next step, is to bring Enzyme in and show that it is differentiable. For this we'll have to allocate the shadow memory that stores the gradient information. 

In [35]:
∂vor_tend = one(vor_tend)
∂vor = zero(vor)
∂damping = zero(damping)
∂damping_impl = zero(damping_impl);

In [38]:
∇! = autodiff(horizontal_diffusion_kernel!(device.device_KA(), device.n))
ev = ∇!(Duplicated(vor_tend, ∂vor_tend), Duplicated(vor, ∂vor), Duplicated(damping, ∂damping), 
    Duplicated(damping_impl, ∂damping_impl); ndrange=length(vor_tend))
wait(ev)

I don't know of an easy way to prove the derivatives are correct, but as Enzyme has the paradigm to through errors rather than incorrect gradients, let's hope for the best.

# How to submit 

Please submit all the functions as pull requests on GitHub into the `kernelabstractions_adtest` branch (not the main branch!), there will be a code review and then we accept it into the branch. 

The functions and for the AD test with Enzyme, we have seperate scripts in the test folder where they should be added 